<a href="https://colab.research.google.com/github/kangning-huang/PopWeightedUHI/blob/main/01_pop_weighted_UHI_timeseries2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import

In [1]:
!pip install geemap &> /dev/null

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from scipy import stats

from tqdm import tqdm

import geemap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
# from ipyleaflet import GeoJSON

import altair as alt
import folium

import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=b_7XAB9Js-Oue0GFezSxqbYFocm3Rr8n9k_BspxBDj8&tc=jPwdptDHVxFQHUF96uxEfqFD4atqj0WeMS7HdLuLPeE&cc=BYowwvVnEDiHKel_sV98iTvW2ZbVVSnujRrtBQv8OCo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qgn4lIQadK6K_TKtagdiE2GTWtGvR7rNNDlW411_PqHRjSc5rb6dKc

Successfully saved authorization token.


In [3]:
!pip install geopandas &> /dev/null

In [4]:
import geopandas as gdp
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

## Load FUAs

In [6]:
gdf_FUAs = gdp.read_file('/content/drive/MyDrive/Research_Projects/2022_UHI_vs_GHG/data/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg')
gdf_FUAs = gdf_FUAs.sort_values('FUA_p_2015', ascending=False)
print(gdf_FUAs.head(5))

gdf_Shanghai = gdf_FUAs.loc[gdf_FUAs['eFUA_name']=='Shanghai']
gdf_Helsinki = gdf_FUAs.loc[gdf_FUAs['eFUA_name']=='Helsinki']
fc_Shanghai = geemap.geopandas_to_ee(gdf_Shanghai)
fc_Helsinki = geemap.geopandas_to_ee(gdf_Helsinki)
Map = geemap.Map()
Map.addLayer(fc_Helsinki,{},'Shanghai')
Map.center_object(fc_Helsinki.geometry())
Map

      eFUA_ID  UC_num                                             UC_IDs  \
5885   5129.0     5.0                      12962;12964;12966;12989;12995   
3789   7466.0     8.0            6955;6964;6993;7003;7063;7080;7085;7108   
5312   4897.0     1.0                                              11862   
2358  10097.0     7.0          12400;12406;12443;12455;12465;12471;12500   
7251   2845.0    14.0  12817;12819;12821;12824;12829;12840;12837;1284...   

                 eFUA_name  Commuting Cntry_ISO   Cntry_name  FUA_area  \
5885                 Tokyo        1.0       JPN        Japan   11971.0   
3789     Delhi [New Delhi]        1.0       IND        India    5569.0   
5312               Jakarta        1.0       IDN    Indonesia    5292.0   
2358              Shanghai        1.0       CHN        China    5878.0   
7251  Quezon City [Manila]        1.0       PHL  Philippines    4873.0   

      UC_area    FUA_p_2015     UC_p_2015    Com_p_2015  \
5885   5582.0  3.647179e+07  3.353853e+

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Load UHIs

In [7]:
# Function to get UHI stats by year
def GetUhiStatsByYear(year):
  # getting the image collection
  uhiModis1 = ee.ImageCollection('users/lorenzomentaschi/uhiModis');
  uhiModis2 = ee.ImageCollection('users/lucfeyen3/uhiModis');
  uhiModis = uhiModis1.merge(uhiModis2);
  # Filter images in the year
  filterYear = ee.Filter.calendarRange(year, year, 'year')
  # Filter pixles with more than 50 rual pixels
  def FilterByMinNRuralPixel(img):
    minNRuralPixel = 50
    ruralPxlCnt = img.select('N_NOTURB_PIX_Day')
    msk = ruralPxlCnt.gte(minNRuralPixel)
    return img.select(['UHI_Day_1km', 'UHI_Night_1km']).updateMask(msk)
  # Apply both filters
  ImColYear = ee.ImageCollection(uhiModis.filter(filterYear).map(FilterByMinNRuralPixel))
  ImStatsYear = ImColYear.reduce(ee.Reducer.percentile([95, 99]))
  return ImStatsYear

# Add test layer to map
visParms_uhi = {
    'min': -18, 'max': 18,
    'palette': ['00008F', '0010FF', '0060FF', '00FFFF', 'CCFFCC', 'FFFF00', 'FF8000', 'FF2000', '800000']
}

Map = geemap.Map()
Map.addLayer(GetUhiStatsByYear(2020).select('UHI_Day_1km_p99'), visParms_uhi, 'UHI 2020')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Load WorldPop

In [8]:
# Function to get WorldPop by year
def GetWorldPopByYear(year):
  ImColPop = ee.ImageCollection("WorldPop/GP/100m/pop")
  ImColPopYear = ImColPop.filter(ee.Filter.calendarRange(year,year,'year'))
  return ImColPopYear.mosaic()

# Add test layer to map
visParms_pop = {
  'bands': ['population'],
  'min': 0.0,
  'max': 50.0,
  'palette': ['24126c', '1fff4f', 'd4ff50']
}

Map = geemap.Map()
Map.addLayer(GetWorldPopByYear(2020), visParms_pop, 'Population 2020')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# UHI times POP

In [9]:
# Function to get UHI times POP
def GetUhiStatsTimesPopByYear(year):
  ImUhiStat = GetUhiStatsByYear(year)
  ImPop = GetWorldPopByYear(year).updateMask(ImUhiStat.select(1))
  return ImUhiStat.multiply(ImPop)

# Add test layer to map
visParms_uhipop = {
    'max': 5000,
    'palette': ['green', 'yellow', 'orange', 'red', 'purple']
}

Map = geemap.Map()
Map.addLayer(GetUhiStatsTimesPopByYear(2020).select('UHI_Day_1km_p99'), visParms_uhipop, 'UHI x POP 2020')
Map.center_object(fc_Shanghai.geometry())
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# Mean UHI, POP for FUA

## Define function to calculate UHI per cap

In [10]:
# Testing
# gdf_fua = gdf_Shanghai
# gdf_fua = gdf_Helsinki
# year = 2020

# Define function to calculate UHI per cap for FUA, in a year
def GetUhiPercapFUAbyYear(gdf_fua, year):
  fc_fua = geemap.geopandas_to_ee(gdf_fua).first()
  geo_fua = fc_fua.geometry()
  uhi_cnt = GetUhiStatsByYear(year).reduceRegion(ee.Reducer.count(), geo_fua, 1000).getInfo()
  if uhi_cnt['UHI_Day_1km_p99'] > 0:
    uhi_mean = GetUhiStatsByYear(year).reduceRegion(ee.Reducer.mean(), geo_fua, 1000).getInfo()
    pop_sum = GetWorldPopByYear(year).reduceRegion(ee.Reducer.sum(), geo_fua, 100).getInfo()
    uhiTimesPop_sum = GetUhiStatsTimesPopByYear(year).reduceRegion(ee.Reducer.sum(), geo_fua, 100).getInfo()
    fua_name = gdf_fua[['eFUA_name']].iloc[0].iloc[0]
    fua_id = gdf_fua[['eFUA_ID']].iloc[0].iloc[0]
    uhi_day_p99_percap = uhiTimesPop_sum['UHI_Day_1km_p99'] / pop_sum['population']
    uhi_nig_p99_percap = uhiTimesPop_sum['UHI_Night_1km_p99'] / pop_sum['population']
    df_UhiPercap = pd.DataFrame({
        'Year': year,
        'UHI_Day_1km_p99': [uhi_mean['UHI_Day_1km_p99']],
        'UHI_Night_1km_99': [uhi_mean['UHI_Night_1km_p99']],
        'UHI_Day_1km_p95_percap': [uhi_day_p99_percap], 
        'UHI_Night_1km_p99_percap': [uhi_nig_p99_percap],
        'eFUA_name':[fua_name], 
        'eFUA_ID':[fua_id]}, index=[fua_id])
  else:
    df_UhiPercap = pd.DataFrame({})
  return df_UhiPercap

print(GetUhiPercapFUAbyYear(gdf_Helsinki, 2020).empty)
print(GetUhiPercapFUAbyYear(gdf_Shanghai, 2020))

True
         Year  UHI_Day_1km_p99  UHI_Night_1km_99  UHI_Day_1km_p95_percap  \
10097.0  2020         5.895528          2.350958                7.805211   

         UHI_Night_1km_p99_percap eFUA_name  eFUA_ID  
10097.0                  3.456066  Shanghai  10097.0  


## Apply function to all FUAs

In [ ]:
path_out = '/content/drive/MyDrive/Research_Projects/2022_UHI_vs_GHG/results/pop_weighted_UHIs/'
filename_out = path_out + 'pop_weighted_uhi_fuas.csv'

df_UhiPercap = pd.DataFrame()

for index, row in tqdm(gdf_FUAs.head(100).iterrows()):
  for year in range(2003, 2021):
    fua_id = row['eFUA_ID']
    gdf_fua = gdf_FUAs.loc[gdf_FUAs['eFUA_ID']==fua_id]
    df_UhiPercap = df_UhiPercap.append(GetUhiPercapFUAbyYear(gdf_fua, year))

df_UhiPercap.head(10)

df_UhiPercap.to_csv(filename_out)

2it [39:06, 1224.69s/it]